In [1]:
import sklearn
import pandas as pd 
import numpy as np
import gc
import time
import copy
from operator import itemgetter
from sklearn.model_selection import KFold


path = "D:/kaggle/MovieLens/"
path2 = 'D:/kaggle/MovieLens/ml-latest-small/'
#data = pd.read_csv( path+"rating.csv", usecols=[0,1])
data = pd.read_csv( path2+"ratings.csv", usecols=[0,1])

In [2]:
print(data.userId.nunique())
print(data.movieId.nunique())
data.max()

610
9724


userId        610
movieId    193609
dtype: int64

In [3]:
class itemCF(object):
    
    def __init__(self, user_items):
        self.sorted_flag = False
        self.user_items = user_items
    
    def ItemSimilarity(self, user_items, is_norm):
        # C为有关系的物品之间的计数
        self.C = dict()
        self.item_count = dict()
        for _, items in user_items.items():
            for i in items:
                self.item_count[i] = self.item_count.get(i, 0) + 1
                for j in items:
                    if i == j:
                        continue
                    if not i in self.C:
                        self.C[i] = {}
                    self.C[i][j] = self.C[i].get(j,0) + 1 
                    # 越活跃的用户，即购买物品越多的用户，所占权重越小
                    # self.C[i][j] = self.C[i].get(j,0) + 1 / np.log(1 + len(items))
    
        # W为有关系的物品之间的相似度
        self.W = self.C
        for i,related_items in self.C.items():
            for j, co_count in related_items.items():
                self.W[i][j] = co_count / np.sqrt(self.item_count[i] * self.item_count[j])
    
        # 归一化能降低热门物品的权重
        if is_norm == False:
            return 1
    
        for i,related_items in self.W.items():
            max_i = np.max(list(related_items.values()))
            for j, w_ij in related_items.items():
                self.W[i][j] = w_ij / max_i
    '''
    input:
        {userA:{item1,item2,item3},
         userB:{item2,item4}}
    '''
    
    def train(self, is_norm=True):
        start = time.time()
        self.ItemSimilarity( self.user_items, is_norm)
        print("finished in ",time.time() - start)
        
    def pre_sort(self, K=10):
        self.item_sorted = {}
        for item in self.W.keys():
            self.item_sorted[item] = sorted(self.W[item].items(), key=itemgetter(1), reverse=True)[0:K]
            
        self.sorted_flag = True
    
    def recommend(self, user, K=10, length=10):
        rank = dict()
        hist_items = self.user_items[user]
        for i in hist_items:
            if self.sorted_flag:
                for j, w_ij in self.item_sorted[i]:
                    if j in hist_items:
                        continue
                    rank[j] =  rank.get(j,0) + w_ij
        
            else:
                for j, w_ij in sorted(self.W[i].items(), key=itemgetter(1), reverse=True)[0:K]:
                    if j in hist_items:
                        continue
                    rank[j] =  rank.get(j,0) + w_ij
        
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[0:length]


In [4]:
class userCF(object):
    
    def __init__(self, user_items):
        self.user_items = user_items
        self.usernum = len(self.user_items)
        self.sorted_flag = False
    
    def simple_UserSimilarity(self, user_items):
        self.usernum = len(user_items)
        self.W = np.zeros((self.usernum, self.usernum))
        for u in user_items.keys():
            for v in user_items.keys():
                if u == v:
                    continue
                self.W[u][v] = len(user_items[u] & user_items[v])
                self.W[u][v] /= np.sqrt(len(user_items[u]) * len(user_items[v]))
                
    def train(self, item_users):
        start = time.time()
        self.UserSimilarity( item_users, self.usernum)
        print("finished in ",time.time() - start)
    
    def UserSimilarity(self, item_users, usernum):
        # C为买了相同物品的用户之间的计数
        self.user_count = np.zeros(usernum+1)
        self.C = [{} for i in range(usernum+1)]
        for _, users in item_users.items():
            for u in users:
                self.user_count[u] += 1
                for v in users:
                    if u == v:
                        continue
                    self.C[u][v] = self.C[u].get(v, 0) + 1            
                    # 改进后的相似度
                    # 一个物品，如果它对应的用户越多，那么它的权重越小
                    # self.C[u][v] = self.C[u].get(v, 0) + 1/np.log(1 + len(users))
    
        #calculate finial similarity matrix self.W
        self.W = self.C
        for u in range(1, usernum+1):
            for v, co_count in self.W[u].items():
                self.W[u][v] = co_count / np.sqrt(self.user_count[u] * self.user_count[v])
    
    def pre_sort(self, K=10):
        self.user_sorted = [ 0 for i in range(self.usernum+1)]
        for u in range(1, self.usernum+1):
            self.user_sorted[u] = sorted(self.W[u].items(), key=itemgetter(1), reverse=True)[0:K]
            
        self.sorted_flag = True
    
    def recommend(self, user, K=10, length=10):
        rank = dict()
        hist_items = self.user_items[user]
        if self.sorted_flag == True:
            for v, w_uv in self.user_sorted[user]:
                for v_item in self.user_items[v]:
                    if v_item in hist_items:
                        continue
                    # w_uv为用户u和用户v的兴趣相似度
                    rank[v_item] = rank.get(v_item,0) + w_uv
            
        else:
            for v, w_uv in sorted(W[user].items(), key=itemgetter(1), reverse=True)[0:K]:
                for v_item in user_items[v]:
                    if v_item in hist_items:
                        continue
                    # w_uv为用户u和用户v的兴趣相似度
                    rank[v_item] = rank.get(v_item,0) + w_uv
        
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[0:length]


In [5]:
def Recall_Precision(obj, test, topN):
    hit = 0
    n_recall = 0
    n_precision = 0
    for user in test.keys():
        true = test[user]
        rank = obj.recommend(user, length=topN)
        rank = {it[0] for it in rank}
        hit += len(true & rank)
        n_recall += len(true)
        n_precision += topN
    return hit / n_recall, hit / n_precision


def Coverage(obj, test, topN):
    recommend_items = set()
    all_items = set()
    for user in test.keys():
        all_items.update( test[user])
        rank = obj.recommend(user, length=topN)
        rank = {it[0] for it in rank}
        recommend_items.update(rank)
    return len(recommend_items) / len(all_items)

#如果推荐出的物品都很热门，说明推荐的新颖度较低，否则说明推荐结果比较新颖
def Popularity(obj, train, topN):
    item_popularity = dict()
    for user, items in train.items():
        for item in items:
            item_popularity[item] = item_popularity.get(item, 0) + 1
    ret = 0
    n = 0
    for user in train.keys():
        rank = obj.recommend(user, length=topN)
        rank = {it[0] for it in rank}
        for item in rank:
            ret += np.log(1 + item_popularity[item])
            n += 1
    
    return ret / n 

In [9]:
kf = KFold(n_splits=8, shuffle=True, random_state=22)

for train_index, valid_index in kf.split(data):
    train = data.iloc[train_index]
    test = data.iloc[valid_index]
    user_items = {}
    for userid, group in train.groupby("userId"):
        user_items[userid] = set( group['movieId'])

    item_users = {}
    for movieid, group in train.groupby("movieId"):
        item_users[movieid] = set( group['userId'])
        
    test_user_items = {}
    for userid, group in test.groupby("userId"):
        test_user_items[userid] = set( group['movieId'])
    
    break

In [11]:
user_cfer = userCF(user_items)
user_cfer.train(item_users)
user_cfer.pre_sort()
print("userCF:")
print("Recall and Precision:", Recall_Precision(user_cfer, test_user_items, 10))
print("Coverage:", Coverage(user_cfer, test_user_items, 10))
print('Popularity:', Popularity(user_cfer, user_items, 10))

item_cfer = itemCF(user_items)
item_cfer.train()
item_cfer.pre_sort()
print("itemCF:")
print("Recall and Precision:", Recall_Precision(item_cfer, test_user_items, 10))
print("Coverage:", Coverage(item_cfer, test_user_items, 10))
print('Popularity:', Popularity(item_cfer, user_items, 10))

finished in  1.9739995002746582
userCF:
Recall and Precision: (0.08845696152320508, 0.18429752066115704)
Coverage: 0.14031668696711327
Popularity: 4.787604344420452
finished in  62.87667417526245
itemCF:
Recall and Precision: (0.0864736215787386, 0.18016528925619835)
Coverage: 0.1456760048721072
Popularity: 4.588692347652099
